<a href="https://colab.research.google.com/github/mantoan-thi/Wire-harness-analysis/blob/main/Comparando_chicotes_Yazaki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 5.2 MB/s 


In [38]:
import pandas as pd
import numpy as np

In [90]:
# Carregando o Datastet
df_circuitos = pd.read_excel('/content/Consulta BOM v15.xlsm',sheet_name='CIRCUITO')
df_componentes = pd.read_excel('/content/Consulta BOM v15.xlsm',sheet_name='COMPONENTE')
df_tubos = pd.read_excel('/content/Consulta BOM v15.xlsm',sheet_name='TUBO')
df_sumarizado = pd.read_excel('/content/Consulta BOM v15.xlsm',sheet_name='SUMARIZADO')

# Análise de circuitos

In [182]:
# Selecionando somentes colunas importantes
if 'OT_DISCRIMINADA' in df_circuitos.columns:
  df_circuitos = df_circuitos[['PROD_TERM', 'OT', 'OT_DISCRIMINADA', 'CIRC_ESQ','CIR_DIR', 'TIPO', 
                              'BITOLA', 'COR', 'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ','ACC_ESQ_1',
                              'COD_OP_01_DIR','TERMINAL_DIR','ACC_DIR_1','SOLDA']]

  # Obtenha os nomes dos índices para os quais a coluna 'OT_DISCRIMINADA' tem o valor SIM

  index_ot_discriminada = df_circuitos[df_circuitos['OT_DISCRIMINADA']=='SIM'].index
  if index_ot_discriminada.shape[0] > 0: # Excluindo esses índices de linha de dataFrame
    df_circuitos.drop(index_ot_discriminada, inplace=True)
    df_circuitos.drop(columns=['OT_DISCRIMINADA'],axis=1, inplace=True)
  df_circuitos.fillna(0,inplace=True)

# Quantidade de circuitos por part number
lista_pn = []
pn_anterior = 0
for pn in df_circuitos['PROD_TERM']:
  if pn_anterior != pn:
    pn_anterior = pn
    qtde = df_circuitos[df_circuitos['PROD_TERM']==pn].shape[0]
    lista_pn.append([pn,qtde])
lista_pn =pd.DataFrame(lista_pn,columns=['Part_number','Qtde'])
lista_pn.sort_values(by=['Qtde'],ascending=False,inplace=True)
lista_pn.reset_index(inplace=True,drop=True)


qtde_pn = lista_pn.shape[0]
menor_pn = lista_pn['Part_number'][-1:].values[0]
maior_pn = lista_pn['Part_number'][0]
Describ = lista_pn.describe()


# Calcular a quantidade de comparações
contador1 =0
contador2 = 0
lista_de_comparacao = []
Total = 0
while contador1 < lista_pn['Part_number'].shape[0]:
  contador2=contador1+1
  while contador2 < lista_pn['Part_number'].shape[0]:
    lista_de_comparacao.append(lista_pn['Part_number'][contador1]+' vs '+lista_pn['Part_number'][contador2])
    contador2+=1
    Total+=1
  contador1+=1
  contador2 = 0
lista_de_comparacao = pd.DataFrame(lista_de_comparacao,columns=['Part Numbers'])

In [183]:
def analisar(list_comp):
  contar1 = 0
  contar2 = 0
  list_comp['Delta'] = 0
  while contar1 < list_comp.shape[0]:
    contar2 = contar1+1
    while contar2 < list_comp.shape[0]:
      if list_comp['PROD_TERM'].iloc[contar1]!=list_comp['PROD_TERM'].iloc[contar2]:
        # Tipo+bit+cor
        tipoA = list_comp['TIPO'].iloc[contar1]+list_comp['BITOLA'].iloc[contar1]+list_comp['COR'].iloc[contar1]
        tipoB = list_comp['TIPO'].iloc[contar2]+list_comp['BITOLA'].iloc[contar2]+list_comp['COR'].iloc[contar2]
        TAE = str(list_comp['COD_OP_01_ESQ'].iloc[contar1])+str(list_comp['TERMINAL_ESQ'].iloc[contar1])+str(list_comp['ACC_ESQ_1'].iloc[contar1])
        TAD = str(list_comp['COD_OP_01_DIR'].iloc[contar1])+str(list_comp['TERMINAL_DIR'].iloc[contar1])+str(list_comp['ACC_DIR_1'].iloc[contar1])
        TBE = str(list_comp['COD_OP_01_ESQ'].iloc[contar2])+str(list_comp['TERMINAL_ESQ'].iloc[contar2])+str(list_comp['ACC_ESQ_1'].iloc[contar2])
        TBD = str(list_comp['COD_OP_01_DIR'].iloc[contar2])+str(list_comp['TERMINAL_DIR'].iloc[contar2])+str(list_comp['ACC_DIR_1'].iloc[contar2])
        COMPA = int(list_comp['COMP'].iloc[contar1])
        COMPB = int(list_comp['COMP'].iloc[contar2])
        dif = abs(COMPA-COMPB)
        JTA = str(list_comp['SOLDA'].iloc[contar1])[0]
        JTB = str(list_comp['SOLDA'].iloc[contar2])[0]
        if tipoA == tipoB:                            # Comparando Tipo,Bit e cor
          if COMPA == COMPB or dif < 50:              # Checando dimensional
            if TAE==TBE or TAD==TBD:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='Comum' 
              list_comp['Delta'].iloc[contar2] ='Comum'
            if TAE==TBD or TAD==TBE:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='Comum' 
              list_comp['Delta'].iloc[contar2] ='Comum'

      contar2 +=1
    contar1+=1
    contar2 = 0
    list_comp['Delta'].loc[list_comp['Delta']==0]='DIF'
    lista_analisada = list_comp.sort_values(by=['OT'],ascending=True,inplace=True)
  return list_comp

In [184]:
# Part number que será o 'PARA'
partA = lista_pn['Part_number'][7]
partA[5:10]

'F5380'

In [189]:
contador = 0
for pn in lista_pn['Part_number']:
  if pn != partA:
    list_comp = df_circuitos[(df_circuitos['PROD_TERM']==pn) | (df_circuitos['PROD_TERM']==partA)]
    nova_lista = analisar(list_comp)
    nova_lista['Total'] = 1
    nova_lista['Delta'].loc[(nova_lista['Delta']=='DIF') & (nova_lista['PROD_TERM']==partA)]='Adicionar'
    nova_lista['Delta'].loc[(nova_lista['Delta']=='DIF') & (nova_lista['PROD_TERM']==pn)]='Excluir'

    nova_lista1 = nova_lista.copy()
    nova_lista2 = nova_lista.copy()
    nova_lista = pd.pivot_table(nova_lista1,index=['OT', 'CIRC_ESQ', 'CIR_DIR', 'TIPO', 'BITOLA', 'COR',
       'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ', 'ACC_ESQ_1', 'COD_OP_01_DIR',
       'TERMINAL_DIR', 'ACC_DIR_1', 'SOLDA','Delta'],values=["Total"],columns=['PROD_TERM'],aggfunc=[np.sum],fill_value=0)
   
    resumo = pd.pivot_table(nova_lista2,index=['PROD_TERM','Delta'],values=["Total"],aggfunc=np.sum)

    # Usando o ExcelWriter, cria um doc .xlsx, usando engine='xlsxwriter'
    writer = pd.ExcelWriter('/content/Analise'+pn[5:10]+' vs '+partA[5:10]+'.xlsx', engine='xlsxwriter')

    # Armazena cada df em uma planilha diferente do mesmo arquivo
    resumo.to_excel(writer, sheet_name='Resumo')
    lista_de_comparacao.to_excel(writer, sheet_name='Lista_comparações')
    lista_pn.to_excel(writer, sheet_name='Part_number_Quant')
    nova_lista.to_excel(writer, sheet_name='Lista_analisada')

    # Fecha o ExcelWriter e gera o arquivo .xlsx
    writer.save()
    break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

# Trabalhando com tabelas

In [161]:
nova_lista[nova_lista['Delta']!='COMUM']

,PROD_TERM,OT,CIRC_ESQ,CIR_DIR,TIPO,BITOLA,COR,COMP,COD_OP_01_ESQ,TERMINAL_ESQ,ACC_ESQ_1,COD_OP_01_DIR,TERMINAL_DIR,ACC_DIR_1,SOLDA,Delta,Total
305,82111F5F30 000,45,#5F 499,B03 013,CIVUS,0.5,G,2971,0,7116-3154-02,0,D,7116-5044-02,7158-3009-60,0,Excluir,1
306,82111F5F30 000,47,#PE 499,B02 014,CIVUS,0.5,L,1974,0,7116-3154-02,0,D,7116-5044-02,7158-3009-60,0,Excluir,1
307,82111F5F30 000,49,%A3 452,Z3K 013,CIVUS,0.5,W-B,613,B4,7409-9915,0,D,7116-5044-02,7158-3009-60,0,Excluir,1
308,82111F5F30 000,51,%A4 453,Z3J 014,CIVUS,0.5,W-B,609,ON,7409-9915,0,D,7116-5044-02,7158-3009-60,0,Excluir,1
314,82111F5F30 000,58,000 000,D3C 027,LFAVSS,0.3,G,1505,D,7116-5044-02,7158-3006-90,0,7017-1180-02,0,1 0056 0056,Excluir,1
318,82111F5F30 000,62,000 000,C30 007,LFAVSS,0.3,B,1887,0,7017-1181-02,0,D,7116-4019-02,7158-3299-10,1 0066 0066,Excluir,1
319,82111F5F30 000,63,%A3 452,D3B 026,LFAVSS,0.3,W-B,462,B4,7409-9915,0,D,7116-5044-02,7158-3006-90,0,Excluir,1
320,82111F5F30 000,64,%A4 453,D3D 027,LFAVSS,0.3,W-B,458,ON,7409-9915,0,D,7116-5044-02,7158-3006-90,0,Excluir,1
321,82111F5F30 000,65,000 000,C31 008,LFAVSS,0.3,B,3036,0,7017-1181-02,0,D,7116-4019-02,7158-3299-10,1 0066 0066,Excluir,1
323,82111F5F30 000,67,%A3 452,Z95 008,LFAVSS,0.3,W-B,918,B4,7409-9915,0,D,7116-4019-02,7158-3299-10,0,Excluir,1


In [181]:
pd.pivot_table(nova_lista,index=['PROD_TERM','Delta'],values=["Total"],aggfunc=np.sum)

KeyError: ignored